# Naïve fusion

In [ ]:
from models import *
from utils import *

In [ ]:
class MultiFusion(nn.Module):
    def __init__(self, eeg_dim, eeg_emb_dim,pos, phy_dim, phy_emb_dim, phy_feat_dim, eeg_feat_dim=5, n_class=2):
        super(MultiFusion, self ).__init__()
        
        self.eeg_attention = RegionRNN_DEAP(eeg_emb_dim//2, 1, eeg_feat_dim, f_dim=eeg_feat_dim)
        self.phy_attention = SimpleRNN_DEAP(phy_emb_dim, 1, phy_feat_dim, f_dim=phy_feat_dim)
        
        self.Classifier = nn.Sequential(
            nn.Linear(eeg_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, n_class),
            nn.Softmax(dim=1)
            )
        
        self.Regressor = nn.Sequential(
            nn.Linear(eeg_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1)
            )
        
        self.Discriminator = nn.Sequential(
            GradientReversal(),
            nn.Linear(eeg_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1),
            nn.Sigmoid()
            )
        
    def forward(self, x_eeg, x_phy):
        b_size = x_eeg.shape[0]
        
        spatial_eeg = x_eeg.transpose(1,2)
        feat_eeg = self.eeg_attention(spatial_eeg)
        
        spatial_phy = x_phy.transpose(1,2)
        feat_phy = self.phy_attention(spatial_phy)
        feat = feat_eeg*feat_phy
                
        return feat

class CatFusion(nn.Module):
    def __init__(self, eeg_dim, eeg_emb_dim,pos, phy_dim, phy_emb_dim, phy_feat_dim, eeg_feat_dim=5, n_class=2):
                
        self.eeg_attention = RegionRNN_DEAP(eeg_emb_dim//2, 1, eeg_feat_dim, f_dim=eeg_feat_dim)
        
        self.phy_attention = nn.Sequential(
                    Flatten(),
                    nn.Linear(5*8, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU(True),
                    nn.Linear(64 ,phy_emb_dim),
                    nn.BatchNorm1d(phy_emb_dim),
                    nn.ReLU(True)
                    )
        self.Classifier = nn.Sequential(
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.BatchNorm1d(64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, n_class),
            nn.Softmax(dim=1)
            )
        
        self.Regressor = nn.Sequential(
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1)
            )
        
        self.Discriminator = nn.Sequential(
            GradientReversal(),
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1),
            nn.Sigmoid()
            )
        
        
    def forward(self, x_eeg, x_phy):
        b_size = x_eeg.shape[0]
        
        spatial_eeg = x_eeg.transpose(1,2)
        feat_eeg = self.eeg_attention(spatial_eeg)
        
        spatial_phy = x_phy.transpose(1,2)
        feat_phy = self.phy_attention(spatial_phy)
        feat = torch.cat([feat_eeg,feat_phy], axis=1)
                
        return feat

class BilFusion(nn.Module):
    def __init__(self, eeg_dim, eeg_emb_dim,pos, phy_dim, phy_emb_dim, phy_feat_dim, eeg_feat_dim=5, n_class=2):
        super(BilFusion, self ).__init__()
        
        self.eeg_attention = RegionRNN_DEAP(eeg_emb_dim//2, 1, eeg_feat_dim, f_dim=eeg_feat_dim)   
        
        self.phy_attention = nn.Sequential(
                    Flatten(),
                    nn.Linear(5*8, 64),
                    nn.BatchNorm1d(64),
                    nn.ReLU(True),
                    nn.Linear(64 ,phy_emb_dim),
                    nn.BatchNorm1d(phy_emb_dim),
                    nn.ReLU(True)
                    )
        if torch.cuda.is_available():
            self.mcb = CompactBilinearPooling(eeg_emb_dim, phy_emb_dim, eeg_emb_dim+phy_emb_dim).cuda()
        else :
            self.mcb = CompactBilinearPooling(eeg_emb_dim, phy_emb_dim, eeg_emb_dim+phy_emb_dim)

        self.Classifier = nn.Sequential(
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, n_class),
            nn.Softmax(dim=1)
            )
        
        self.Regressor = nn.Sequential(
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1)
            )
        
        self.Discriminator = nn.Sequential(
            GradientReversal(),
            nn.Linear(eeg_emb_dim+phy_emb_dim, 64),
            nn.Dropout(0.25),
            nn.ReLU(True),
            nn.Linear(64, 1),
            nn.Sigmoid()
            )
        
    def forward(self, x_eeg, x_phy):
        b_size = x_eeg.shape[0]
        
        spatial_eeg = x_eeg.transpose(1,2)
        feat_eeg = self.eeg_attention(spatial_eeg)
        
        spatial_phy = x_phy.transpose(1,2)
        feat_phy = self.phy_attention(spatial_phy)
        
        feat = self.mcb(feat_eeg, feat_phy)
                
        return feat

In [ ]:
'''Parameters'''
batch_size = 64
n_epoch = 1000

In [ ]:
def train_model(feat_eeg, feat_phy, label, participant, path_results, training_info, n_class, pos, fusion):
    EEG = EEGPhiDataset(label=label, eeg=feat_eeg, phi=feat_phy)
    Tot = {}
    session = np.load('dataset/seed_iv/session.npy')
    for p in tqdm(np.unique(participant)):
        idx = np.argwhere(participant==p).squeeze()
        np.random.shuffle(idx)
        id_train = idx[:int(0.8*len(idx))]
        id_test = idx[int(0.8*len(idx)):]
        
        np.random.shuffle(id_train)
        np.random.shuffle(id_test)
        Test = Subset(EEG, id_test)
        Train = Subset(EEG, id_train)
        
        Trainloader = DataLoader(Train, batch_size=batch_size, shuffle=False, pin_memory=False)
        Testloader = DataLoader(Test, batch_size=batch_size, shuffle=False, pin_memory=False)
        
        n_chan = feat_eeg.shape[2]
        f_dim_eeg = feat_eeg.shape[1]
        
        phy_dim = feat_phy.shape[2]
        f_dim_phy = feat_phy.shape[1]
        
        if fusion == 'cat':
            net = CatFusion(eeg_dim=n_chan, eeg_emb_dim=64, eeg_feat_dim=f_dim_eeg, pos=pos, phy_dim=phy_dim, phy_emb_dim=16, phy_feat_dim=f_dim_phy, n_class=n_class).cuda()
        elif fusion == 'mult':
            net = MultiFusion(eeg_dim=n_chan, eeg_emb_dim=64, eeg_feat_dim=f_dim_eeg, pos=pos, phy_dim=phy_dim, phy_emb_dim=64, phy_feat_dim=f_dim_phy, n_class=n_class).cuda()
        elif fusion == 'bil':
            net = BilFusion(eeg_dim=n_chan, eeg_emb_dim=64, eeg_feat_dim=f_dim_eeg, pos=pos, phy_dim=phy_dim, phy_emb_dim=16, phy_feat_dim=f_dim_phy, n_class=n_class).cuda()
        
        optimizer = optim.SGD(net.parameters(), lr=0.0005, momentum=0.9, weight_decay=0.005)
        
        writer = SummaryWriter('runs/rnn_'+fusion+'_p_'+str(p))
        
        res = []
        for epoch in range(n_epoch):
            
            running_loss = 0.0
            evaluation = []
            t_cycle = iter(cycle(Testloader))
            for i, data in enumerate(Trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                inputs_eeg_source, inputs_phy_source, labels = data
                del data
                
                data = next(t_cycle)
                inputs_eeg_test, inputs_phy_test, _ = data
                del data
                
                domain = torch.cat([torch.ones(inputs_eeg_source.shape[0]),
                                   torch.zeros(inputs_eeg_test.shape[0])]).cuda()
                inputs_eeg = torch.cat([inputs_eeg_source, inputs_eeg_test])
                inputs_phy = torch.cat([inputs_phy_source, inputs_phy_test])
                
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                
                feat_ = net(inputs_eeg.to(torch.float32).cuda(), inputs_phy.to(torch.float32).cuda())
                
                domain_pred = net.Discriminator(feat_).squeeze()
                
                loss = torch.nn.functional.binary_cross_entropy(domain_pred, domain)
                outputs = net.Classifier(feat_[:inputs_eeg_source.shape[0]])

                label_loss = torch.nn.functional.cross_entropy(outputs, labels.to(torch.long).cuda())
                loss += label_loss
                loss.backward()
                optimizer.step()

                _, predicted = torch.max(outputs, 1)
                num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
                mean = num_of_true/labels.shape[0]
                running_loss = label_loss.item()
                evaluation.append(mean)
            running_loss = running_loss/(i+1)
            running_acc = sum(evaluation)/len(evaluation)
            
            validation_loss = 0.0
            validation_acc = 0.0
            evaluation = []
            for i, data in enumerate(Testloader, 0):
                inputs_eeg, inputs_phy, labels = data
                del data
                feat_ = net(inputs_eeg.to(torch.float32).cuda(), inputs_phy.to(torch.float32).cuda())
                outputs = net.Classifier(feat_)
                loss = torch.nn.functional.cross_entropy(outputs, labels.cuda())
                validation_loss += loss.item()
                
                _, predicted = torch.max(outputs, 1)
                num_of_true = torch.sum(predicted.detach().cpu()==labels).numpy()
                evaluation.append(num_of_true/labels.shape[0])
            validation_loss = validation_loss/(i+1)
            validation_acc = sum(evaluation)/len(evaluation)
            
            writer.add_scalar('Loss/Train', running_loss, epoch)
            writer.add_scalar('Loss/Test', validation_loss, epoch)
            writer.add_scalar('Acc/Train', running_acc, epoch)
            writer.add_scalar('Acc/Test', validation_acc, epoch)
            
            res.append((running_loss, running_acc, validation_loss, validation_acc))
        Tot['participant_'+str(p)] = np.asarray(res)
        np.save(os.path.join(path_results, training_info), Tot)

## PhyDAA Dataset

In [ ]:
'''Load File'''
label = np.load('dataset/phydaa/label.npy').astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/phydaa/participant.npy')
elec_pos = np.load('information/phydaa_eeg.npy')

### Multiplication

In [ ]:
train_model(feat_eeg=np.load('dataset/phydaa/feat_eeg.npy'), feat_phy=np.expand_dims(np.load('dataset/phydaa/feat_phy.npy'),1), label=label, 
            participant=participant, path_results='res/ind/', training_info='phydaa_mult', n_class=n_class, pos=elec_pos, fusion='mult')

### Concatenation

In [ ]:
train_model(feat_eeg=np.load('dataset/phydaa/feat_eeg.npy'), feat_phy=np.expand_dims(np.load('dataset/phydaa/feat_phy.npy'),1), label=label, 
            participant=participant, path_results='res/ind/', training_info='phydaa_cat', n_class=n_class, pos=elec_pos, fusion='cat')

### Compact Bilinear Pooling

In [ ]:
train_model(feat_eeg=np.load('dataset/phydaa/feat_eeg.npy'), feat_phy=np.expand_dims(np.load('dataset/phydaa/feat_phy.npy'),1), label=label, 
            participant=participant, path_results='res/ind/', training_info='phydaa_bil', n_class=n_class, pos=elec_pos, fusion='bil')

## SEED IV

In [ ]:
'''Load File'''
label = np.load('dataset/seed_iv/label.npy').astype(int)
n_class = len(np.unique(label))
participant = np.load('dataset/seed_iv/participant.npy')
elec_pos = np.load('information/seed_iv_eeg.npy')

### Multiplication

In [ ]:
train_model(feat_eeg=np.load('dataset/seed_iv/feat_eeg.npy'), feat_phy=np.expand_dims(np.load('dataset/seed_iv/feat_phy.npy'),1), label=label, 
            participant=participant, path_results='res/dep/', training_info='seed_iv_mult', n_class=n_class, pos=elec_pos, fusion='mult')

### Concatenation'''
        

In [ ]:
phy_feat = np.load('dataset/seed_iv/feat_phy.npy')
phy_feat[:, :12] = (phy_feat[:, :12] - phy_feat[:, :12].min())/np.max(phy_feat[:, :12] - phy_feat[:, :12].min()) 
phy_feat[:, 12:16] = (phy_feat[:, 12:16] - phy_feat[:, 12:16].min())/np.max(phy_feat[:, 12:16] - phy_feat[:, 12:16].min()) 
phy_feat[:, 16:18] = (phy_feat[:, 16:18] - phy_feat[:, 16:18].min())/np.max(phy_feat[:, 16:18] - phy_feat[:, 16:18].min()) 
phy_feat[:, 18:22] = (phy_feat[:, 18:22] - phy_feat[:, 18:22].min())/np.max(phy_feat[:, 18:22] - phy_feat[:, 18:22].min())

train_model(feat_eeg=np.load('dataset/seed_iv/feat_eeg.npy'), feat_phy=np.expand_dims(phy_feat,1)[:, :, :22], label=label, 
            participant=participant, path_results='res/dep/', training_info='seed_iv_cat_rnn', n_class=n_class, pos=elec_pos, fusion='cat')

### Compact Bilinear Pooling

In [ ]:
phy_feat = np.load('dataset/seed_iv/feat_phy.npy')
phy_feat[:, :12] = (phy_feat[:, :12] - phy_feat[:, :12].min())/np.max(phy_feat[:, :12] - phy_feat[:, :12].min()) 
phy_feat[:, 12:16] = (phy_feat[:, 12:16] - phy_feat[:, 12:16].min())/np.max(phy_feat[:, 12:16] - phy_feat[:, 12:16].min()) 
phy_feat[:, 16:18] = (phy_feat[:, 16:18] - phy_feat[:, 16:18].min())/np.max(phy_feat[:, 16:18] - phy_feat[:, 16:18].min()) 
phy_feat[:, 18:22] = (phy_feat[:, 18:22] - phy_feat[:, 18:22].min())/np.max(phy_feat[:, 18:22] - phy_feat[:, 18:22].min())

train_model(feat_eeg=np.load('dataset/seed_iv/feat_eeg.npy'), feat_phy=np.expand_dims(phy_feat,1)[:, :, :22], label=label, 
            participant=participant, path_results='res/dep/', training_info='seed_iv_bil_rnn', n_class=n_class, pos=elec_pos, fusion='bil')

## DEAP

In [ ]:
'''Load File'''
label = np.load('dataset/deap/label.npy') - 4.5
n_class = 2 #len(np.unique(label))
participant = np.load('dataset/deap/participant.npy')

### Multiplication

In [ ]:
train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 0]>0).astype(int), 
            participant=participant, path_results='res/dep/', training_info='deap_mult_valence', n_class=n_class, pos=None, fusion='mult')

train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 1]>0).astype(int), 
            participant=participant, path_results='res/dep/', training_info='deap_mult_arousal', n_class=n_class, pos=None, fusion='mult')

### Concatenation

In [ ]:
train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 0]>0).astype(int), 
            participant=participant, path_results='res/ind/', training_info='deap_cat_valence', n_class=n_class, pos=None, fusion='cat')

train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 1]>0).astype(int), 
            participant=participant, path_results='res/ind/', training_info='deap_cat_arousal', n_class=n_class, pos=None, fusion='cat')

### Compact Bilinear Pooling

In [ ]:
train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 0]>0).astype(int), 
            participant=participant, path_results='res/dep/', training_info='deap_bil_valence', n_class=n_class, pos=None, fusion='bil')

train_model(feat_eeg=np.load('dataset/deap/feat_eeg.npy'), feat_phy=np.load('dataset/deap/feat_phy.npy'), label=(label[:, 1]>0).astype(int), 
            participant=participant, path_results='res/dep/', training_info='deap_bil_arousal', n_class=n_class, pos=None, fusion='bil')

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)